In [1]:
import pandas as pd
import numpy as np

# ------------------------------------------------------------------
# 1)  FILE IMPORTS
# ------------------------------------------------------------------
ac2b = pd.read_csv("https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/accomodations_to_venues.csv", index_col=0)   # A → Bus
tmat = pd.read_csv("https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/time_matrix.csv",            index_col=0)   # A + V ↔ A + V
v2b_raw = pd.read_csv("https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/main/matrixes/venues_to_bus_matrix.csv", header=None)   # messy header


# Unify the 3 Matrix

In [2]:

# ------------------------------------------------------------------
# 2)  CLEAN THE “VENUE → BUS” MATRIX
# ------------------------------------------------------------------
bus_ids = v2b_raw.iloc[0].astype(str).tolist()        # first row = 69 bus IDs
blank_rows = v2b_raw.index[v2b_raw.isna().all(axis=1)].tolist()

v2b = (
    v2b_raw
    .drop(index=[0] + blank_rows)                     # drop header + blank lines
    .reset_index(drop=True)
)
v2b.columns = bus_ids
v2b.index   = [f"V{i+1}" for i in range(len(v2b))]    # 32 venues → V1 … V32
v2b = v2b.astype(float)


In [3]:

# ------------------------------------------------------------------
# 3)  COLLECT *EVERY* NODE LABEL
# ------------------------------------------------------------------
accommodations = sorted({i for i in ac2b.index}.union({i for i in tmat.index if i.startswith("A")}))
venues         = sorted({i for i in tmat.index if i.startswith("V")})        # V1 … V34
bus_stops      = sorted(bus_ids)                                             # 69 bus IDs

all_nodes = accommodations + venues + bus_stops

# ------------------------------------------------------------------
# 4)  BUILD AN EMPTY SQUARE MATRIX AND DROP THE BLOCKS IN
# ------------------------------------------------------------------
merged = pd.DataFrame(np.nan, index=all_nodes, columns=all_nodes, dtype=float)

# A ↔ Bus
merged.loc[ac2b.index,   ac2b.columns] = ac2b.values
merged.loc[ac2b.columns, ac2b.index ] = ac2b.values.T

# V ↔ Bus
merged.loc[v2b.index,    v2b.columns]  = v2b.values
merged.loc[v2b.columns,  v2b.index ]  = v2b.values.T

# A/V ↔ A/V
merged.loc[tmat.index,   tmat.columns] = tmat.values
merged.loc[tmat.columns, tmat.index ] = tmat.values.T

# ------------------------------------------------------------------
# 5)  COPY ONE-SIDED VALUES + ZERO THE DIAGONAL
# ------------------------------------------------------------------
merged.update(merged.T)                 # if only one direction is known, copy it
np.fill_diagonal(merged.values, 0)

# ------------------------------------------------------------------
# 6)  SAVE THE RESULT
# ------------------------------------------------------------------
merged.to_csv("merged_matrix.csv", index=True)
print("Merged matrix shape:", merged.shape)
print("File written → merged_matrix.csv")

Merged matrix shape: (154, 154)
File written → merged_matrix.csv


In [4]:
merged

,A1,A10,A11,A12,A13,A14,A15,A16,A17,A18,...,BT19,BT20,BT21,BT22,BT23,BT24,BT25,BT26,BT27,BT28
A1,0.0,3304.0,3496.0,1727.0,1956.0,2020.0,2410.0,1512.0,3273.0,3098.0,...,1850.0,2149.0,1860.0,2759.0,2024.0,2357.0,2234.0,1594.0,1485.0,1745.0
A10,3263.0,0.0,4901.0,2745.0,2974.0,3033.0,2460.0,2917.0,2778.0,3209.0,...,1696.0,1627.0,2491.0,2459.0,2890.0,2960.0,2575.0,2406.0,2420.0,2006.0
A11,3459.0,4958.0,0.0,3305.0,3534.0,3598.0,4079.0,2432.0,4852.0,4676.0,...,3477.0,3757.0,3064.0,4313.0,3578.0,3911.0,3788.0,3326.0,3408.0,3477.0
A12,1753.0,2784.0,3117.0,0.0,593.0,1014.0,1617.0,1405.0,2390.0,2214.0,...,1556.0,1785.0,2037.0,1875.0,1140.0,1473.0,1350.0,1783.0,1424.0,1630.0
A13,1909.0,2939.0,3272.0,576.0,0.0,489.0,1654.0,1561.0,2426.0,2250.0,...,1711.0,1940.0,2193.0,1911.0,1176.0,1509.0,1386.0,1938.0,1579.0,1785.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BT24,2357.0,2960.0,3911.0,1473.0,1509.0,1455.0,1027.0,2040.0,1565.0,1326.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
BT25,2234.0,2575.0,3788.0,1350.0,1386.0,1332.0,250.0,1918.0,1442.0,1234.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
BT26,1594.0,2406.0,3326.0,1783.0,1938.0,2233.0,1496.0,1365.0,2523.0,2315.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
BT27,1485.0,2420.0,3408.0,1424.0,1579.0,1874.0,1556.0,1447.0,2584.0,2376.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN


# Add Nan Values (b->b)

In [ ]:
import googlemaps
from datetime import datetime

# Initialize the Google Maps client with your API key
gmaps = googlemaps.Client(key='key')

# Build list of origin and destination tuples using the dataframe coordinates
origins = bus_terminals[['Latitude', 'Longitude']].apply(lambda row: (row['Latitude'], row['Longitude']), axis=1).tolist()
destinations = bus_terminals[['Latitude', 'Longitude']].apply(lambda row: (row['Latitude'], row['Longitude']), axis=1).tolist()

# Request the distance matrix in batches to avoid MAX_DIMENSIONS_EXCEEDED error
max_elements = 100
max_destinations_per_request = 25  # Free API limit for destinations per request
destination_batch_size = max_destinations_per_request
origin_batch_size = max_elements // destination_batch_size

# Initialize an empty results matrix
results = [[None] * len(destinations) for _ in range(len(origins))]

for i in range(0, len(origins), origin_batch_size):
    origin_chunk = origins[i:i+origin_batch_size]
    for j in range(0, len(destinations), destination_batch_size):
        destination_chunk = destinations[j:j+destination_batch_size]
        response = gmaps.distance_matrix(origin_chunk, destination_chunk, mode='driving')
        for index_in_chunk, row in enumerate(response['rows']):
            result_row = i + index_in_chunk
            for dest_index, element in enumerate(row['elements']):
                if element['status'] == 'OK':
                    results[result_row][j+dest_index] = element['duration']['value']
                else:
                    results[result_row][j+dest_index] = None

b2b = pd.DataFrame(results, index=bus_terminals['id'], columns=bus_terminals['id'])
b2b

id,BT25,BT07,BT13,BT10,BD01,BD02,BD05,BD06,BD09,BD11,...,BL22,BL23,BT26,BL19,BT04,BT19,BL03,BL07,BT05,BT15
id,,,,,,,,,,,,,,,,,,,,,
BT25,0,925,1392,595,1156,1274,2406,488,906,1342,...,2763,1895,1345,966,958,1186,1224,1133,1190,1241
BT07,893,0,826,1036,449,604,1820,1169,200,747,...,2519,1131,853,526,370,481,635,576,624,534
BT13,1513,1105,0,1656,1411,1240,1734,1788,1089,1323,...,2330,1861,496,1071,920,1152,1525,1114,785,1430
BT10,669,1004,1471,0,1272,1354,2485,900,1124,1421,...,3125,1974,1424,1045,1037,1265,937,1212,1269,1320
BD01,1377,489,1103,1280,0,636,1899,1652,402,827,...,2599,1211,1163,715,717,498,676,655,901,621
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BT19,1126,537,1151,1270,534,236,1580,1402,458,507,...,2279,858,933,587,580,0,723,335,949,552
BL03,1280,711,1324,910,673,829,2045,1556,676,973,...,2745,1356,1323,1119,939,707,0,801,1122,755
BL07,983,563,1052,1127,721,307,1450,1259,563,336,...,2149,1039,790,444,437,350,827,0,838,656


# merge it to get final matrix

In [15]:
merged.loc[b2b.index, b2b.columns] = b2b.values
merged.loc[b2b.columns, b2b.index] = b2b.values.T

In [16]:
merged

,A1,A10,A11,A12,A13,A14,A15,A16,A17,A18,...,BT19,BT20,BT21,BT22,BT23,BT24,BT25,BT26,BT27,BT28
A1,0.0,3304.0,3496.0,1727.0,1956.0,2020.0,2410.0,1512.0,3273.0,3098.0,...,1850.0,2149.0,1860.0,2759.0,2024.0,2357.0,2234.0,1594.0,1485.0,1745.0
A10,3263.0,0.0,4901.0,2745.0,2974.0,3033.0,2460.0,2917.0,2778.0,3209.0,...,1696.0,1627.0,2491.0,2459.0,2890.0,2960.0,2575.0,2406.0,2420.0,2006.0
A11,3459.0,4958.0,0.0,3305.0,3534.0,3598.0,4079.0,2432.0,4852.0,4676.0,...,3477.0,3757.0,3064.0,4313.0,3578.0,3911.0,3788.0,3326.0,3408.0,3477.0
A12,1753.0,2784.0,3117.0,0.0,593.0,1014.0,1617.0,1405.0,2390.0,2214.0,...,1556.0,1785.0,2037.0,1875.0,1140.0,1473.0,1350.0,1783.0,1424.0,1630.0
A13,1909.0,2939.0,3272.0,576.0,0.0,489.0,1654.0,1561.0,2426.0,2250.0,...,1711.0,1940.0,2193.0,1911.0,1176.0,1509.0,1386.0,1938.0,1579.0,1785.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BT24,2357.0,2960.0,3911.0,1473.0,1509.0,1455.0,1027.0,2040.0,1565.0,1326.0,...,1847.0,1958.0,2368.0,1327.0,980.0,0.0,934.0,1965.0,1990.0,1634.0
BT25,2234.0,2575.0,3788.0,1350.0,1386.0,1332.0,250.0,1918.0,1442.0,1234.0,...,1126.0,1238.0,1647.0,840.0,857.0,952.0,0.0,1244.0,1479.0,914.0
BT26,1594.0,2406.0,3326.0,1783.0,1938.0,2233.0,1496.0,1365.0,2523.0,2315.0,...,933.0,1044.0,1418.0,1854.0,2018.0,2087.0,1345.0,0.0,357.0,707.0
BT27,1485.0,2420.0,3408.0,1424.0,1579.0,1874.0,1556.0,1447.0,2584.0,2376.0,...,1164.0,1311.0,1731.0,1914.0,1635.0,2047.0,1405.0,362.0,0.0,987.0


In [ ]:
merged.to_csv("merged_matrix.csv", index=True)
print("Merged matrix shape:", merged.shape)
print("File written → merged_matrix.csv")

Merged matrix shape: (154, 154)
File written → merged_matrix.csv
